In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from imblearn.over_sampling import SMOTE
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import re 

In [2]:
data=pd.read_csv(r"C:\Users\Zidan Ahmed\Documents\training.1600000.processed.noemoticon.csv",encoding='iso-8859-1',header=None)

In [3]:
data


,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [4]:
print(data.columns)

Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')


In [5]:
data.describe()

,0,1
count,1.600000e+06,1.600000e+06
mean,2.000000e+00,1.998818e+09
std,2.000001e+00,1.935761e+08
min,0.000000e+00,1.467810e+09
25%,0.000000e+00,1.956916e+09
50%,2.000000e+00,2.002102e+09
75%,4.000000e+00,2.177059e+09
max,4.000000e+00,2.329206e+09


In [6]:
data[0].unique()

array([0, 4], dtype=int64)

In [7]:
df=data.drop([1,2,3,4],axis=1)

In [53]:
df

,0,5
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [11]:
df=df.rename(columns={0:'Label',5:'Text'})

In [12]:
df

,Label,Text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,1,Just woke up. Having no school is the best fee...
1599996,1,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,Happy 38th Birthday to my boo of alll time!!! ...


# Data preprocessing

In [13]:
df=df.replace(4,1)  # Changing the value of postive that is 4 to 1 

In [14]:
df

,Label,Text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,1,Just woke up. Having no school is the best fee...
1599996,1,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,Happy 38th Birthday to my boo of alll time!!! ...


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   Label   1600000 non-null  int64 
 1   Text    1600000 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [16]:
import spacy 
import nltk
nltk.download('wordnet')
nlp=spacy.load('en_core_web_sm')
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package wordnet to C:\Users\Zidan
[nltk_data]     Ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

url_mention_pattern = re.compile(r"(?:\@|https?\://)\S+|[^\w\s#]")

In [18]:
def preprocess_tweet(tweet):
    # Convert to lowercase
    tweet = tweet.lower()
    # Remove URLs, mentions, and non-alphanumeric characters except hashtags
    tweet = url_mention_pattern.sub('', tweet)
    # Tokenize by simple whitespace
    tokens = tweet.split()
    # Lemmatize and filter out stopwords and short tokens
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and len(token) > 1]
    # Join tokens back into a processed tweet
    processed_tweet = ' '.join(lemmatized_tokens)
    return processed_tweet
df['Preprocessed_text']=[preprocess_tweet(txt) for txt in df['Text']]
df['word_length']=df['Preprocessed_text'].apply(lambda x: len(x.split(" ")))
df['char_length']=df['Preprocessed_text'].apply(lambda x: len(x))

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [29]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Example DataFrame (assuming you have a DataFrame `df` with columns 'Preprocessed_text' and 'Label')
# df = pd.DataFrame({'Preprocessed_text': ['example text 1', 'example text 2'], 'Label': [1, 0]})

# Tokenize and pad sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Preprocessed_text'])
sequences = tokenizer.texts_to_sequences(df['Preprocessed_text'])
word_index = tokenizer.word_index

# Padding sequences
max_len = 100  # Maximum length of sequences
data = pad_sequences(sequences, maxlen=max_len)

# Labels for sentiment analysis (assuming you have binary labels in df['Label'])
labels = df['Label'].values

# Model definition
vocab_size = len(word_index) + 1  # Vocabulary size
embed_dim = 50  # Embedding dimension

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embed_dim, input_length=max_len))
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model.add(Dropout(0.5))
model.add(LSTM(units=32))
model.add(Dense(units=1, activation='sigmoid'))

# Build the model explicitly by specifying input shape
model.build(input_shape=(None, max_len))

print(model.summary())

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True)

# Training the model
history = model.fit(data, labels, epochs=1,
                    batch_size=32, validation_split=0.2, callbacks=[early_stopping, checkpoint])

# Save the final model
model.save('my_model.keras')

# Making predictions
# Example: predicting on new data
test_text = ["This movie is great!", "I didn't like this film."]
test_sequences = tokenizer.texts_to_sequences(test_text)
test_data = pad_sequences(test_sequences, maxlen=max_len)

predictions = model.predict(test_data)
print("Predictions:")
print(predictions)

# Load the saved model and make predictions again
loaded_model = tf.keras.models.load_model('my_model.keras')
loaded_predictions = loaded_model.predict(test_data)
print("Loaded Model Predictions:")
print(loaded_predictions)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ (None, 100, 50)             │      21,884,750 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_6 (Bidirectional)      │ (None, 100, 128)            │          58,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 100, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_14 (LSTM)                       │ (None, 32)                  │          20,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,964,271 (83.79 MB)

 Trainable params: 21,964,271 (83.79 MB)

 Non-trainable params: 0 (0.00 B)

None
40000/40000 ━━━━━━━━━━━━━━━━━━━━ 14508s 363ms/step - accuracy: 0.7841 - loss: 0.4535 - val_accuracy: 0.6834 - val_loss: 0.6452
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 550ms/step
Predictions:
[[0.84637016]
 [0.42888835]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 502ms/step
Loaded Model Predictions:
[[0.84637016]
 [0.42888835]]


In [30]:
import pickle

# Save the tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
